In [31]:
import os, pyperclip, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
import subprocess
from smms import *
from utils import *
crf=18.0
qcomp=0.7
aq_mode=3
aq_s=0.7
bitdepth=8
newline="\r\n"

# PROCUCT_DEST_DIR=r"/home/qianyi/Downloads/"
PROCUCT_DEST_DIR=r"/BT/"
# MOVIE=r"/BT/The.Villainess.2017.1080p.Blu-ray.REMUX.DTS-PTH.mkv"

#https://api.rhilip.info/ptgen.html
DOUBAN_ID="30391739"

SRC="Light from Light 2019 1080p Blu-ray AVC DTS-HD MA 5.1-UNRELiABLE".replace(" ",".")
REMUX_DIRNAME="Light.from.Light.2019.Blu-ray.1080p.AVC.REMUX.DTS-HD.MA-PTH"
ENCODE_FILENAME="Light.from.Light.2019.Blu-ray.1080p.REMUX.DTS-PTH.mkv"
MOVIE=f"{PROCUCT_DEST_DIR}/{REMUX_DIRNAME}/{REMUX_DIRNAME}.mkv"

ACKNOWLEDGEMENT='''原盘来自UNRELiABLE，字幕取自字幕库
'''

# SOUNDTRACK=""
SOUNDTRACK="【英语】【简英|英字幕】"
# SOUNDTRACK="【韩语】【简|繁|简英|繁英|英字幕】"

SUBTITLE_GZ=1 #官字组

LOG_DIR="."

In [32]:
metainfo=getmetainfo(DOUBAN_ID)
smms = SMMS('hzlclock', 'Hezl1996')
smms.get_api_token()

In [25]:
# crop=subprocess.getoutput("ffmpeg -i "+MOVIE+" -t 1 -vf cropdetect -f null - 2>&1")
#ffmpeg -i $MOVIE -t 1 -vf cropdetect -f null - 2>&1 | awk '/crop/ { print $NF }' | tail -1
# try: crop="-filter:v \""+re.findall(r'crop=.*',crop)[-1]+"\""
# except: pass
# MOVIE=r"/sdcard/Freaky.2020.1080p.BluRay.REMUX.DTS-louco@PTH.mkv"

#deinterlace: yadif
#denoise: hqdn3d
#crop: crop=in_w-480:in_h,
crop='-filter:v "hqdn3d=4.0"'
test=""
#--demuxer y4m --preset slower --profile high --level 4.1
x264_stmt=f"x264  --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

# --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower
# --pools 12
x265_stmt=f"x265 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --pools 36 \
--rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"

# test="-ss 3700 -t 60"

_args=x264_stmt.replace(" --","\n --").split("\n")[1:-1]
args=[]
for i in _args:
    newitem=i.strip().replace("--","").replace(" ","=").replace(":",",")
    if "=" not in newitem:
        newitem+="=1"
    args.append(newitem)
# args=[i.strip().replace("--","").replace(" ","=").replace(":",",") for i in args]

_args5=x265_stmt.replace(" --","\n --").split("\n")[1:-1]
args5=[]
for i in _args5:
    newitem=i.strip().replace("--","").replace(" ","=").replace(":",",")
    if "=" not in newitem:
        newitem+="=1"
    args5.append(newitem)
        
# args5=[i.strip().replace("--","").replace(" ","=").replace(":",",") for i in args5]

# x264_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x264")
x264_filename=ENCODE_FILENAME.replace("REMUX", "x264")
x264_dirname=x264_filename.replace(".mkv","")
x264_fullpath=os.path.join(PROCUCT_DEST_DIR, x264_dirname, x264_filename)
# x265_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")
x265_filename=ENCODE_FILENAME.replace("REMUX", "x265")
x265_dirname=x265_filename.replace(".mkv","")
x265_fullpath=os.path.join(PROCUCT_DEST_DIR, x265_dirname, x265_filename)

cmd264=f"mkdir -p {os.path.join(PROCUCT_DEST_DIR, x264_dirname)}\n"
cmd265=f"mkdir -p {os.path.join(PROCUCT_DEST_DIR, x265_dirname)}\n"

torrent264=f"transmission-create -p -o {PROCUCT_DEST_DIR}/{x264_dirname}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, x264_dirname)}"
torrent265=f"transmission-create -p -o {PROCUCT_DEST_DIR}/{x265_dirname}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, x265_dirname)}"

cmd264+=f"ffmpeg -c:a dca -core_only true {test} -i {MOVIE} -c:v libx264 \
-profile:v high {crop} -preset slower -x264-params {':'.join(args)} \
-max_muxing_queue_size 1024 -bsf:a dca_core -c:a copy -c:s copy -max_muxing_queue_size 1024 -map 0 -map -v -map V {x264_fullpath} 2>&1|\
tee {LOG_DIR}/x264.log\nmkvpropedit {x264_fullpath} --add-track-statistics-tags\n"

cmd265+=f"ffmpeg -c:a dca -core_only true {test} -i {MOVIE} \
-pix_fmt yuv420p10le -c:v libx265 -profile:v main10 {crop} -preset slower -x265-params {':'.join(args5)} \
-max_muxing_queue_size 1024 -bsf:a dca_core -c:a copy -c:s copy -max_muxing_queue_size 1024 -map 0 -map -v -map V {x265_fullpath} 2>&1|\
tee {LOG_DIR}/x265.log\nmkvpropedit {x265_fullpath} --add-track-statistics-tags\n"

if len(test)==0:
    cmd264+=get_captures("x264",x264_fullpath)+"\n"
    cmd264+=torrent264+"\n"
    cmd265+=get_captures("x265",x265_fullpath)+"\n"
    cmd265+=torrent265+"\n"
    
remux_fullpath=f"{PROCUCT_DEST_DIR}/{REMUX_DIRNAME}/{REMUX_DIRNAME}.mkv"
cmdremux=get_captures("REMUX",remux_fullpath)
cmdremux+=f"transmission-create -p -o {PROCUCT_DEST_DIR}/{REMUX_DIRNAME}.torrent -t https://www.pthome.net/announce.php \
{os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME)}"

with open("x264.sh","w") as f:
    f.write(cmd264)
with open("x265.sh","w") as f:
    f.write(cmd265)
with open("remux.sh","w") as f:
    f.write(cmdremux)

In [27]:
encode_picbed=upload_folder("x264_captures",smms)

===UPLOAD===
https://i.loli.net/2021/02/19/XC56sFdAtqbvUwW.jpg
https://i.loli.net/2021/02/19/iK5AYrStfMIcWyN.jpg
https://i.loli.net/2021/02/19/acnRr4UumjiAQMP.jpg
https://i.loli.net/2021/02/19/jDKTMXqWVRl81u4.jpg
https://i.loli.net/2021/02/19/PXY8EvUhOWlrFuN.jpg
https://i.loli.net/2021/02/19/Y3aBKFlur4etH1C.jpg


In [28]:
remux_picbed=upload_folder("REMUX_captures",smms)

===UPLOAD===
https://i.loli.net/2021/02/19/uaRV4mMrQpT9Kf2.jpg
https://i.loli.net/2021/02/19/C1In6i9h7X52vpo.jpg
https://i.loli.net/2021/02/19/rwo47JBZaHf6AtX.jpg
https://i.loli.net/2021/02/19/IjxJobvEKNgGQsA.jpg
https://i.loli.net/2021/02/19/5UHA3qGyVoKTXef.jpg
https://i.loli.net/2021/02/19/EiMHT2jUgDbxOJo.jpg


In [33]:
# del get_bbcode
with open(f"x264.bbcode.txt","w") as f:
    f.write(get_bbcode(x264_fullpath, src=SRC, logname="x264.log", picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))
with open(f"x265.bbcode.txt","w") as f:
    f.write(get_bbcode(x265_fullpath, src=SRC, logname="x265.log", picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))
with open(f"remux.bbcode.txt","w") as f:
    f.write(get_bbcode(remux_fullpath, src=SRC, picbed=encode_picbed, metainfo=metainfo,ack=ACKNOWLEDGEMENT))

Title: /BT/Light from Light 2019 Blu-ray 1080p x264 DTS-PTH/Light from Light 2019 Blu-ray 1080p x264 DTS-PTH mkv
Title: /BT/Light from Light 2019 Blu-ray 1080p x265 DTS-PTH/Light from Light 2019 Blu-ray 1080p x265 DTS-PTH mkv
Title: /BT//Light from Light 2019 Blu-ray 1080p AVC REMUX DTS-HD MA-PTH/Light from Light 2019 Blu-ray 1080p AVC REMUX DTS-HD MA-PTH mkv


In [34]:
with open(f"x264.bbcode.base64.txt","w") as f:
    f.write(getbase64json(x264_fullpath, src=SRC, logname="x264.log",\
                          picbed=encode_picbed, metainfo=metainfo, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
with open(f"x265.bbcode.base64.txt","w") as f:
    f.write(getbase64json(x265_fullpath, src=SRC, logname="x265.log",\
                          picbed=encode_picbed, metainfo=metainfo, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
with open(f"remux.bbcode.base64.txt","w") as f:
    f.write(getbase64json(remux_fullpath, src=SRC, logname="",\
                          picbed=remux_picbed, metainfo=metainfo, is_remux=True, tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))

Title: /BT/Light from Light 2019 Blu-ray 1080p x264 DTS-PTH/Light from Light 2019 Blu-ray 1080p x264 DTS-PTH mkv
Title: /BT/Light from Light 2019 Blu-ray 1080p x265 DTS-PTH/Light from Light 2019 Blu-ray 1080p x265 DTS-PTH mkv
Title: /BT//Light from Light 2019 Blu-ray 1080p AVC REMUX DTS-HD MA-PTH/Light from Light 2019 Blu-ray 1080p AVC REMUX DTS-HD MA-PTH mkv
